In [3]:
import requests
import pandas as pd
import numpy as np

In [4]:
url = 'http://127.0.0.1:25510/list/roots?sec=STOCK'
headers = {'Accept': 'application/json'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    # The request was successful
    json_data = response.json()
    print(json_data)
else:
    # Request failed
    print(f"Request failed with status code {response.status_code}")
    print(response.text)

{'header': {'id': 1, 'latency_ms': 156, 'error_type': 'null', 'error_msg': 'null', 'next_page': 'null', 'format': None}, 'response': ['A', 'AA', 'AAA', 'AAAP', 'AAAU', 'AABA', 'AAC', 'AAC.U', 'AAC.WS', 'AACC', 'AACG', 'AACI', 'AACIU', 'AACIW', 'AACOU', 'AACOW', 'AACQ', 'AACQU', 'AACQW', 'AACT', 'AACT.U', 'AACT.WS', 'AADI', 'AADR', 'AAIC', 'AAIC.PR.B', 'AAIC.PR.C', 'AAIN', 'AAIR', 'AAIT', 'AAL', 'AALCP', 'AAM.PR.A', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPB', 'AAPC', 'AAPD', 'AAPH', 'AAPL', 'AAPU', 'AAPY', 'AAQC', 'AAQC.U', 'AAQC.WS', 'AAT', 'AATC', 'AAU', 'AAVL', 'AAWW', 'AAXJ', 'AAXN', 'AB', 'ABAC', 'ABAT', 'ABAX', 'ABB', 'ABBB', 'ABBV', 'ABC', 'ABCB', 'ABCD', 'ABCL', 'ABCM', 'ABCO', 'ABCW', 'ABDC', 'ABEO', 'ABEOW', 'ABEQ', 'ABEV', 'ABFS', 'ABG', 'ABGB', 'ABGI', 'ABHD', 'ABIL', 'ABILW', 'ABIO', 'ABL', 'ABLLW', 'ABLV', 'ABLVW', 'ABLX', 'ABM', 'ABMD', 'ABNB', 'ABOS', 'ABR', 'ABR.PR.E', 'ABSI', 'ABST', 'ABT', 'ABTL', 'ABTX', 'ABUS', 'ABVA', 'ABVC', 'ABVX', 'ABY', 'AC', 'ACA', '

In [5]:
from enum import Enum

class Req(Enum):
    TRADE = 'trade'
    QUOTE = 'quote'
    OI = 'open_interest'

class Right(Enum):
    CALL = 'C'
    PUT = 'P'

In [6]:
class ThetaDataAPI:
    def __init__(self):
        self.base_url = 'http://127.0.0.1:25510/'

    def _get_req(self, url: str, headers: dict, params: dict=None):
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Request failed with status code {response.status_code}")
            print(response.text)

    def get_roots(self, security_type: str='OPTION'):
        url = f'{self.base_url}list/roots?sec={security_type}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)

    def get_expirations(self, root:str):
        url = f'{self.base_url}list/expirations?root={root}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_strikes(self, root: str, exp: str):
        url = f'{self.base_url}list/strikes?root={root}&exp={exp}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_dates(self, root: str, security_type: str="option", exp: str=None, strike_right: tuple[str, Right]=None):
        url = f'{self.base_url}list/dates/{security_type}/quote'
        url_params = {}
        url_params["root"] = root
        if exp is not None:
            url_params["exp"] = exp
        if strike_right is not None:
            url_params["strike"] = strike_right[0]
            url_params["right"] = strike_right[1].value
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers, params=url_params)
    
    """
    NOTE this function has a 750ms overhead 
    """
    def get_contracts(self, date: str, req: Req=Req.TRADE):
        url = f'{self.base_url}list/contracts/option/{req.value}?start_date={date}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
        
    def get_eod_prices(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right):
        url = f'{self.base_url}hist/option/eod?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)

    def get_quotes(self, root: str, start_date: str, end_date: str, exp: str, strike: str, right: Right, ivl: str=None):
        url = f'{self.base_url}hist/option/quote?root={root}&start_date={start_date}&end_date={end_date}&strike={strike}&exp={exp}&right={right.value}&ivl={0 if ivl is None else ivl}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)

        



In [7]:
api = ThetaDataAPI()
api.get_expirations(root='AAPL')

{'header': {'id': 2,
  'latency_ms': 24,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': None},
 'response': [20120601,
  20120608,
  20120616,
  20120622,
  20120629,
  20120706,
  20120713,
  20120721,
  20120727,
  20120803,
  20120810,
  20120818,
  20120824,
  20120831,
  20120907,
  20120914,
  20120922,
  20120928,
  20121005,
  20121012,
  20121020,
  20121026,
  20121102,
  20121109,
  20121117,
  20121123,
  20121130,
  20121207,
  20121214,
  20121222,
  20121228,
  20130104,
  20130111,
  20130119,
  20130125,
  20130201,
  20130208,
  20130216,
  20130222,
  20130301,
  20130308,
  20130316,
  20130322,
  20130328,
  20130405,
  20130412,
  20130420,
  20130426,
  20130503,
  20130510,
  20130518,
  20130524,
  20130531,
  20130607,
  20130614,
  20130622,
  20130628,
  20130705,
  20130712,
  20130720,
  20130726,
  20130802,
  20130809,
  20130817,
  20130823,
  20130830,
  20130906,
  20130913,
  20130921,
  20130927,
  20131004,
  2013

In [9]:
client = ThetaDataAPI()
root = "AAPL"
expiries = client.get_expirations(root=root)["response"]
exp = expiries[len(expiries)-50]
print(expiries[-50:])
strikes = client.get_strikes(root=root, exp=exp)
print(strikes)
dates = client.get_dates(root=root, exp=exp)
print(dates)
"""
contracts = client.get_contracts(date=20220512)
print(contracts["header"])
print(contracts["response"][:10])
"""
eod = client.get_eod_prices(root=root, start_date=20230301, end_date=20230401, exp=20230406, strike=150000, right=Right.CALL)
print(eod)
quotes = client.get_quotes(root=root, start_date=20230331, end_date=20230331, exp=20230406, strike=150000, right=Right.CALL)
print(quotes)


[20230406, 20230414, 20230421, 20230428, 20230505, 20230512, 20230519, 20230526, 20230602, 20230609, 20230616, 20230623, 20230630, 20230707, 20230714, 20230721, 20230728, 20230804, 20230811, 20230818, 20230825, 20230901, 20230908, 20230915, 20230922, 20230929, 20231006, 20231013, 20231020, 20231027, 20231103, 20231110, 20231117, 20231124, 20231201, 20231208, 20231215, 20231222, 20240119, 20240216, 20240315, 20240419, 20240621, 20240920, 20241220, 20250117, 20250620, 20250919, 20251219, 20260116]
{'header': {'id': 9, 'latency_ms': 24, 'error_type': 'null', 'error_msg': 'null', 'next_page': 'null', 'format': None}, 'response': [50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 105000, 110000, 115000, 120000, 125000, 130000, 131000, 132000, 133000, 134000, 135000, 136000, 137000, 138000, 139000, 140000, 141000, 142000, 143000, 144000, 145000, 146000, 147000, 148000, 149000, 150000, 152500, 155000, 157500, 160000, 162500, 165000, 167500, 170000, 172500, 175000, 